# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import boto3
import psycopg2
import configparser
import pandas as pd


In [11]:
from IPython.display import FileLink, FileLinks

### Step 1: Scope the Project and Gather Data

#### Scope 
The aim of this project is the development of a Datawarehouse for US immigration data. The DWH is designed for correlation of data of incoming immigrants, their origin countries and destination cities demographics. This will allow for Business Intelligence reports.


#### Describe and Gather Data 
The datasets used in this project are provided by Udacity for the capstone project. The datasets are:

* I94 Immigration Data: This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace.
* U.S. City Demographic Data: This data comes from OpenSoft.
* Airport Code Table: This is a simple table of airport codes and corresponding cities.

Udacity also provides a world temperature dataset which was not used in this project, because the I94 data is from the year 2016, and the temperature dataset contains data up to 2013. Therefore, no conclusions can be obtained from associating this data to the immigration data. 

In [4]:
# Airport Code Table dataset view
df_airport = pd.read_csv("airport-codes_csv.csv")
display(df_airport.info())
display(df_airport.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


None

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [7]:
# US City Demographics dataset view
df_city_demographics = pd.read_csv("us-cities-demographics.csv", delimiter=";")
display(df_city_demographics.info())
display(df_city_demographics.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


None

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [2]:
# I94 Immigration dataset analysis, for the month of April
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_i94 = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [3]:
display(df_i94.info())
display(df_i94.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096313 entries, 0 to 3096312
Data columns (total 28 columns):
cicid       float64
i94yr       float64
i94mon      float64
i94cit      float64
i94res      float64
i94port     object
arrdate     float64
i94mode     float64
i94addr     object
depdate     float64
i94bir      float64
i94visa     float64
count       float64
dtadfile    object
visapost    object
occup       object
entdepa     object
entdepd     object
entdepu     object
matflag     object
biryear     float64
dtaddto     object
gender      object
insnum      object
airline     object
admnum      float64
fltno       object
visatype    object
dtypes: float64(13), object(15)
memory usage: 661.4+ MB


None

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.



This analysis will only be focused on the columns that uniquely identify each row, for each dataset. The resto of the columns will not be analysed, because empty or duplicate values will tolerated for this project and may be handled at a higher layer, i.e., on queries designed for specific use cases.

__Airport Codes Table dataset analysis__

In [9]:
# 'ident' uniquely identifies each airport. We will check for duplicate and null values
print("Unique Aiport Codes present in column 'ident': {}".format(df_airport.ident.isna().sum()))
print("Duplicate Aiport Codes present in column 'ident': {}".format(len(df_airport.ident)-len(df_airport.ident.drop_duplicates())))

Unique Aiport Codes present in column 'ident': 0
Duplicate Aiport Codes present in column 'ident': 0


__US City Demographics dataset analysis__

In [8]:
# 'City' uniquely identifies each airport. We will check for duplicate and null values
print("Unique Cities present in column 'City': {}".format(df_city_demographics.City.isna().sum()))
print("Duplicate Cities present in column 'City': {}".format(len(df_city_demographics.City)-len(df_city_demographics.City.drop_duplicates())))

Unique Cities present in column 'City': 0
Duplicate Cities present in column 'City': 2324


This dataset has a high number of duplicate entries because the same Cities are represented multiples times to display information of ethnical population. 

In Amazon Redshift, when transfering staging data from S3 to final tables, ethnical population count is merged into the same line and new columns are created. The new created columns are the count of the following population ethnical groups: white, black, asian, native and latino.

In the following chapters this procedure can be consulted.


__i94 Immigration dataset analysis__

In [5]:
# 'cicid' uniquely identifies each i94 immigration record. We will check for duplicate and null values
print("Unique Record IDs present in column 'cicid': {}".format(df_i94.cicid.isna().sum()))
print("Duplicate Record IDs present in column 'cicid': {}".format(len(df_i94.cicid)-len(df_i94.cicid.drop_duplicates())))

Unique Aiport Codes present in column 'ident': 0
Duplicate Aiport Codes present in column 'ident': 0


#### Cleaning Steps
First, raw data was loaded to the following staging tables in Redshift, from the CSV files:
* Staging_airport_codes: Airport code table
* Staging_demographics: US City Demographic dataset
* Staging_country_codes: Country code dataset obtained from the provided SAS Labels Description
* Staging_i94_data: I94 immigration dataset

The datasets and converted to the appropriate format in the final dimension and fact tables, when loaded from staging tables.
**************************
Document steps necessary to clean the data

In [ ]:
# The following SQL queries were used to load data from staging to the dimension and fact tables

# Countries data was not didn't need any special arrangement
countries_dim_table_insert = ("""
    SELECT code, country
    FROM staging_country_codes;
    """)


# Cities demograpghics table was redesigned. In the raw data, Cities had multiple duplicate entries with 
# exception of population race data. Therefore, new columns were created in the Cities Demographics dimensional table
# to store this data without the need of duplicate entries of the same city.
cities_demographics_dim_table_insert = ("""
    SELECT      city,
                state,
                median_age,
                male_pop,
                female_pop,
                total_pop,
                veterans,
                foreign_born,
                avg_house_size,
                MAX(CASE WHEN race = 'White' THEN count ELSE 0 END) white_pop,
                MAX(CASE WHEN race = 'Asian' THEN count ELSE 0 END) asian_pop,
                MAX(CASE WHEN race = 'Black or African-American' THEN count ELSE 0 END) black_pop,
                MAX(CASE WHEN race = 'American Indian and Alaska Native' THEN count ELSE 0 END) native_pop,
                MAX(CASE WHEN race = 'Hispanic or Latino' THEN count ELSE 0 END) latino_pop        
    FROM        staging_demographics
    GROUP BY    city,
                state,
                median_age,
                male_pop,
                female_pop,
                total_pop,
                veterans,
                foreign_born,
                avg_house_size
    ;
    """)

# Immigration data was loaded to the arrivals fact table. Date fields were converted from SAS date format to 
# YYYY-MM-DD format. Also, the 'dtaddto' was a string that had to be verified and converted in Redshift when loaded
# to the fact table
arrivals_fact_table_insert = ("""
    SELECT      cicid,
                i94yr,
                i94mon,
                admnum,
                date ('1960-01-01'::date + arrdate * interval '1 day') as arrdate,
                i94res,
                i94port,
                i94visa,
                i94mode,
                i94bir,
                gender,
                CASE
                    WHEN (dtaddto BETWEEN 00000000 AND 12312999) THEN to_date(dtaddto::varchar,'MMDDYYYY')
                    ELSE NULL
                END,
                date ('1960-01-01'::date + depdate * interval '1 day')
    FROM        staging_i94_data
    WHERE       i94res IS NOT NULL;
    """)





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model



The used data model is the following:
<img src="images/data_model.png" width="1000">

It was choosen a Star Schema for this data warehouse, in order to simplify the queries and for easier maintenance.
* Fact Table
 * arrivals: holds the factual and quantifiable data associated to immigrant entry in the US.
 
 
* Dimension Tables:
 * us_airports: holds the data for each airport where immigrants entry to the US.
 * cities_demographics: it stores demographics data for US cities.
 * countries: stores the country codes used in i94 forms, and the corresponding country name.
 * arrival_modes: it stores the arrival modes of the transportation used by the immigrants (e.g.: air, sea).
 * visa_types: stores the visa types assigned to immigrants (e.g.: business, pleasure).

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

The designed data pipeline is the following:
<img src="images/data_pipeline.png" width="1200"/>

The main data pipeline has the following stages:
* staging_tables_create: The staging tables are created to store raw data from source CSV files in S3.
* stage_target table: The staging tables are loaded with the copy command from source S3 csv files.
* dim_"tables"_create: The dimension tables are created in Redshift.
* Load_"target_table"_dim_table: The dimension tables are populated with data from staging tables. Some data cleaning can happen in this stage (e.g.: null or invalid values are handled).
* fact_tables_create: The fact table is created in Redshift.
* Load_"target_table"_fact_table: The fact table is populated with data from staging tables. Some data cleaning and conversion (e.g.: dates) are handled at this stage
* Run_quality_checks: Some quality checks are done to the fact and dimension tables, to ensure the data pipeline was executed successfully.


A manual procedure to upload the source files to S3 is done before running the data pipeline. The following is done:
* i94 immigration SAS data is converted to CSV usign pandas, because Redshift doesn't support SAS file format.
* "i94_country_codes.csv" file is created and copied manually from the provided data dictionary file "I94_SAS_Labels_Descriptions.SAS". This is done because the file was not possible to be loaded to pandas due to SAS format errors. This file contains valid Country Codes used in i94 immigration dataset fields.
* All CSV files were uploaded manually to S3 from this project's workspace.

Below is the code used:

In [ ]:
# S3 boto3 client configuration with Access and Secret Keys
s3 = boto3.client(
    's3',
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
bucket_name = 'dchainho-udacity-dend'

In [ ]:
# CSV files upload to S3 using boto3
s3.upload_file(
    "us-cities-demographics.csv", 
    bucket_name, 
    "capstone/dataset/us-cities-demographics.csv")
s3.upload_file(
    "airport-codes_csv.csv", 
    bucket_name, 
    "capstone/dataset/airport-codes_csv.csv")
s3.upload_file(
    "i94_country_codes.csv", 
    bucket_name, 
    "capstone/dataset/i94_country_codes.csv")

In [ ]:
# i94 pandas file saved to CSV file and upload to S3
df_i94.to_csv("i94_csv/apr16.csv", index=False)
s3.upload_file("i94_csv/apr16.csv", bucket_name, "capstone/dataset/i94/csv/apr16.csv")

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

The data pipeline was built in Airflow as follows:

* dend-capstone.py: Main airflow config file where the DAGs and data pipeline are created.
* stage_redshift.py: Custom operator to create staging DAGs.
* load_dimension.py: Custom operator to create DAGs to load dimension tables.
* load_fact.py: Custom operator to create DAG to load fact table.
* data_quality.py: Custom operator for data quality checks DAGs.
* sql_queries.py: Helper file containing all the SQL queries used in the Redshift.

The files are located in 'airflow/' folder and sub-folders, where they can consulted in depth.

The data pipeline definition can be seen on the code snippet below:

In [ ]:
start_operator = DummyOperator(task_id='Begin_execution',  dag=dag)

# Staging tables creation in Redshift
staging_tables_create = PostgresOperator(
    dag=dag,
    task_id='staging_tables_create',
    postgres_conn_id='redshift',
    sql=SqlQueries.staging_tables_create,
    autocommit=True
)

# Staging raw data from csv files in S3 to Redshift staging tables
stage_airports_to_redshift = StageToRedshiftOperator(
    task_id='Stage_airports',    
    dag=dag,
    redshift_conn_id="redshift",
    table="staging_airport_codes",
    s3_bucket="dchainho-udacity-dend",
    s3_file="capstone/dataset/airport-codes_csv.csv",
    iam_role=IAM_ROLE
)

stage_demographics_to_redshift = StageToRedshiftOperator(
    task_id='Stage_demographics',    
    dag=dag,
    redshift_conn_id="redshift",
    table="staging_demographics",
    s3_bucket="dchainho-udacity-dend",
    s3_file="capstone/dataset/us-cities-demographics.csv",
    iam_role=IAM_ROLE,
    delimiter=";"
)

stage_i94_data_to_redshift = StageToRedshiftOperator(
    task_id='Stage_i94_data',    
    dag=dag,
    redshift_conn_id="redshift",
    table="staging_i94_data",
    s3_bucket="dchainho-udacity-dend",
    s3_file="capstone/dataset/i94/csv/",
    iam_role=IAM_ROLE
)

stage_countries_to_redshift = StageToRedshiftOperator(
    task_id='Stage_countries',    
    dag=dag,
    redshift_conn_id="redshift",
    table="staging_country_codes",
    s3_bucket="dchainho-udacity-dend",
    s3_file="capstone/dataset/i94_country_codes.csv",
    iam_role=IAM_ROLE
)

# Dimension tables creation in Redshift
dim_tables_create = PostgresOperator(
    dag=dag,
    task_id='dim_tables_create',
    postgres_conn_id='redshift',
    sql=SqlQueries.dim_tables_create,
    autocommit=True
)

# Inserting data in dimensional tables, from the staging tables in Redshift
load_visa_types_dim_table = PostgresOperator(
    dag=dag,
    task_id='Load_visa_types_dim_table',
    ui_color = '#358140',
    postgres_conn_id='redshift',
    sql=SqlQueries.load_visa_types,
    autocommit=True
)

load_arrival_modes_dim_table = PostgresOperator(
    dag=dag,
    task_id='Load_arrival_modes_dim_table',
    ui_color = '#358140',
    postgres_conn_id='redshift',
    sql=SqlQueries.load_arrival_modes,
    autocommit=True
)

load_countries_dim_table = LoadDimensionOperator(
    task_id='Load_countries_dim_table',
    dag=dag,
    redshift_conn_id="redshift",
    table="countries",    
    insert_sql_select=SqlQueries.countries_dim_table_insert
)

load_cities_demographics_dim_table = LoadDimensionOperator(
    task_id='Load_cities_demographics_dim_table',
    dag=dag,
    redshift_conn_id="redshift",
    table="cities_demographics",    
    insert_sql_select=SqlQueries.cities_demographics_dim_table_insert
)

load_us_airports_dim_table = LoadDimensionOperator(
    task_id='Load_us_airports_dim_table',
    dag=dag,
    redshift_conn_id="redshift",
    table="us_airports",    
    insert_sql_select=SqlQueries.us_airports_dim_table_insert
)

# Facts table creation
fact_tables_create = PostgresOperator(
    dag=dag,
    task_id='fact_tables_create',
    postgres_conn_id='redshift',
    sql=SqlQueries.fact_tables_create,
    autocommit=True
)

# Loading facts table with data from staging table
load_arrivals_fact_table = LoadFactOperator(
    task_id='Load_arrivals_fact_table',
    dag=dag,
    redshift_conn_id="redshift",
    table="arrivals",    
    insert_sql_select=SqlQueries.arrivals_fact_table_insert
)

# Quality checks on final dimensional and facts table
run_quality_checks_dim_tables = DataQualityOperator(
    task_id='Run_quality_checks_dim_tables',
    redshift_conn_id="redshift",
    tables=DIM_TABLES,
    dag=dag
)

run_quality_checks_fact_tables = DataQualityOperator(
    task_id='Run_quality_checks_fact_tables',
    redshift_conn_id="redshift",
    tables=FACT_TABLES,
    dag=dag
)

end_operator = DummyOperator(task_id='Stop_execution',  dag=dag)


stage_tables_list = [stage_countries_to_redshift,
                     stage_airports_to_redshift, 
                     stage_demographics_to_redshift, 
                     stage_i94_data_to_redshift]

load_dim_tables_list = [load_countries_dim_table, 
                        load_visa_types_dim_table, 
                        load_arrival_modes_dim_table,
                        load_cities_demographics_dim_table]

# Data pipeline creation
start_operator >> staging_tables_create 
staging_tables_create >> stage_tables_list >> dim_tables_create
dim_tables_create >> load_dim_tables_list
load_cities_demographics_dim_table >> load_us_airports_dim_table 
load_dim_tables_list >> fact_tables_create
load_us_airports_dim_table >> fact_tables_create
fact_tables_create >> load_arrivals_fact_table
load_arrivals_fact_table >> [run_quality_checks_dim_tables, run_quality_checks_fact_tables] >> end_operator

#### 4.2 Data Quality Checks
At the end, 6 quality checks are done, by counting the rows of all 6 created tables. If any table has zero count, an error is raised.
The code of the customer operator to create the DAGs can be seen below:

In [ ]:
# DAGs created in the data pipeline
run_quality_checks_dim_tables = DataQualityOperator(
    task_id='Run_quality_checks_dim_tables',
    redshift_conn_id="redshift",
    tables=DIM_TABLES,
    dag=dag
)

run_quality_checks_fact_tables = DataQualityOperator(
    task_id='Run_quality_checks_fact_tables',
    redshift_conn_id="redshift",
    tables=FACT_TABLES,
    dag=dag
)

In [ ]:
# Custom operator DAG count procedure
def execute(self, context):
        redshift = PostgresHook(postgres_conn_id=self.redshift_conn_id)  

        for t in self.tables:
            logging.info(f"Data quality check -> table '{t}' -> verifying row count.")
            sql_query=DataQualityOperator.count_rows.format(t)        
            records = redshift.get_records(sql_query)

            if len(records) < 1 or len(records[0]) < 1:
                raise ValueError(f"Data quality check -> table '{t}' -> FAILED, returned no results.")
            num_records = records[0][0]
            if num_records < 1:
                raise ValueError(f"Data quality check -> table '{t}' -> FAILED, contained 0 rows.")
            logging.info(f"Data quality check -> table '{t}' -> PASSED, with {records[0][0]} records.")


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Dimensional Tables

<img src="images/us_airports.png" width="600">

<img src="images/cities_demographics.png" width="600">

<img src="images/countries.png" width="600">

<img src="images/visa_types.png" width="600">

<img src="images/arrival_modes.png" width="600">

#### Fact Tables

<img src="images/arrivals.png" width="600">

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

#### Clearly state the rationale for the choice of tools and technologies for the project.

The tools used in this project were:
- Airflow: For defining and executing the data pipeline. Airflow was chosen because it provides a highly customizable and programmatic way to build workflows for data processing. It also provides a solid way to automate queries for future needs.
- Amazon S3: To store the raw data files to be used in Amazon Redshift. A cloud solution, very cost efficient, and it provides fast loading of raw data to staging tables in Redshift.
- Amazon Redshift: A cloud service provided by Amazon to host SQL databases. This technology was used because it provides a way for fast scaling, being a cost effective solution and it has fast queries that together with SQL, provides a powerful for joining tables for customizable queries.

#### Propose how often the data should be updated and why.

The Data Warehouse in Reshift could be updated at the end of each day, with the data from all the immigrant arrivals in that period. This can be easily done with Airflow by scheduling DAGs and run the data pipeline automatically.

#### Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x
  * Amazon Redshift is a powerful cloud solution highly scalable. For this project, only 1 cluster was used. If the data was 100x bigger, I would use a higer number of clusters in order to support and parallelize the increased number of queries in order to maintain reduced latencies.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
  * A solution used in this project is already provided for this scenario. Airflow is an excellent tool to run scheduled automatic workflows. In this case, a DAG should be created to update the tables with the previous day new arrivals, every day at, for example, 2am, for all the tables to be finished in time to be used by the dashboard at 7am every day.
 * The database needed to be accessed by 100+ people.
  * Amazon Redshift probably has the capacity to handle the increase query load. Imagining a scenario of 100 users and 20 daily customized queries per use, this would ammount to a total of 2000 queries per day. Dimensioning for a peak simultaneous querying of 75%, i.e., 1500 simultaneous and different queries, generating 1500 temporary tables, Amazon Redshift would be more than enough. From Amazon Redshift site, "The maximum number of tables is 9,900 for large and xlarge cluster node types and 20,000 for 8xlarge cluster node types.". The number of queries could increase 10x or 20x times and still Redshift would be capable of supporting such load.